# Python GIS & Remote Sensing Tutorial

Comprehensive tutorial combining: Python basics, data reading, statistics, visualization, vector (shapefile) analysis, raster analysis, digital image handling, stacking, mosaicking, classification, change detection, and visualization.

Run cells in order. Replace file path placeholders (e.g., `'your_file.tif'`) with your files.

## 1. Python Basics (quick refresher)
Small snippets to remind how cells, printing, variables, lists, functions work.

In [ ]:
# Example: print and simple function
print('Hello, Ahmadullah!')

def add(a, b):
    return a + b

print('2 + 3 =', add(2,3))

## 2. Data Reading (CSV, Excel, Shapefile, GeoTIFF)
Replace paths with your file names.

In [ ]:
import pandas as pd
import geopandas as gpd

# CSV
# df = pd.read_csv('data.csv')
# print(df.head())

# Excel
# df_x = pd.read_excel('data.xlsx', sheet_name=0)
# print(df_x.head())

# Shapefile (GeoPandas)
# shp = gpd.read_file('your_shapefile.shp')
# print(shp.head())
# print(shp.crs)

# Raster (rasterio) - see later raster section

## 3. Basic Statistics (pandas / numpy / statistics)

In [ ]:
import numpy as np
import statistics as stats

# Sample DataFrame
import pandas as pd
df = pd.DataFrame({'Score':[85,90,88,92,76,95],'Age':[20,21,19,22,20,23]})
print(df)
print('\nDescribe:')
print(df.describe())

scores = df['Score']
print('\nMean:', np.mean(scores))
print('Median:', np.median(scores))
print('Mode:', stats.mode(list(scores)))
print('Std dev:', np.std(scores, ddof=1))

## 4. Data Visualization (matplotlib & seaborn)
Examples of line, bar, scatter, histogram, heatmap.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sample
years = [2018,2019,2020,2021,2022]
rain = [2100,2500,1900,2700,3000]
temp = [30.2,29.8,31.1,30.7,29.9]

plt.plot(years, rain, marker='o')
plt.title('Annual Rainfall')
plt.xlabel('Year')
plt.ylabel('Rainfall (mm)')
plt.grid(True)
plt.show()

sns.histplot(rain, bins=5, kde=True)
plt.title('Rainfall Distribution')
plt.show()

# Correlation heatmap example
df_sample = pd.DataFrame({'Rain':rain,'Temp':temp})
sns.heatmap(df_sample.corr(), annot=True)
plt.show()

## 5. Shapefile Analysis (GeoPandas)
Read, inspect, reproject, calculate area/perimeter, attribute & spatial queries, export.

In [ ]:
import geopandas as gpd

# Read shapefile
# shp = gpd.read_file('districts.shp')
# print(shp.head())
# print('CRS:', shp.crs)

# Reproject to UTM (example EPSG:32646)
# shp_utm = shp.to_crs(epsg=32646)
# shp_utm['area_m2'] = shp_utm.geometry.area
# shp_utm['perimeter_m'] = shp_utm.geometry.length
# print(shp_utm[['area_m2','perimeter_m']].head())

# Select by attribute
# dhaka = shp[shp['District']=='Dhaka']
# dhaka.plot()

# Spatial overlay example
# rivers = gpd.read_file('rivers.shp')
# rivers_in_dhaka = gpd.overlay(rivers, dhaka, how='intersection')
# rivers_in_dhaka.plot()

## 6. Raster Analysis (Rasterio + NumPy)
Read raster, show, statistics, masking nodata, simple calculations, save outputs.

In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np

# Open raster
a = rasterio.open('elevation.tif')
print('Width, Height, Bands:', a.width, a.height, a.count)
print('CRS:', a.crs)

band1 = a.read(1).astype('float32')
# Mask nodata if available
if a.nodata is not None:
    band1 = np.where(band1==a.nodata, np.nan, band1)

print('min, max, mean:', np.nanmin(band1), np.nanmax(band1), np.nanmean(band1))

# Display
show(band1, title='Band 1')

# Save processed (example: convert to km)
# profile = a.profile
# profile.update(dtype=rasterio.float32)
# with rasterio.open('elevation_km.tif', 'w', **profile) as dst:
#     dst.write((band1/1000).astype(rasterio.float32), 1)

## 7. Digital Image Handling (multiband images, composites, contrast, indices, filtering)

In [ ]:
from skimage import exposure, filters
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# Read multiband (example Landsat stack)
# stack = rasterio.open('landsat_stack.tif')
# img = stack.read().astype('float32')  # shape: (bands, rows, cols)

# True color composite (example indices)
# rgb = np.dstack((img[3], img[2], img[1]))
# rgb_norm = rgb / np.nanmax(rgb)
# plt.imshow(rgb_norm)
# plt.title('True Color')
# plt.axis('off')
# plt.show()

# NDVI example (NIR=band5, Red=band4)
# nir = img[4]
# red = img[3]
# ndvi = (nir - red) / (nir + red + 1e-9)
# plt.imshow(ndvi, cmap='RdYlGn')
# plt.colorbar(); plt.title('NDVI')
# plt.show()

# Contrast stretching
# p2, p98 = np.percentile(rgb, (2,98))
# rgb_stretched = exposure.rescale_intensity(rgb, in_range=(p2,p98))
# plt.imshow(rgb_stretched/np.nanmax(rgb_stretched)); plt.title('Contrast Stretched')
# plt.show()

# Edge detection example (Sobel) - on single band
# edges = filters.sobel(img[3])
# plt.imshow(edges, cmap='gray'); plt.title('Edges'); plt.axis('off'); plt.show()

## 8. Layer stacking, Mosaic, Subset (mask/clip)
Stack bands, mosaic tiles, and clip by AOI shapefile.

In [ ]:
import glob
from rasterio.merge import merge
from rasterio.mask import mask

# Layer stacking (if separate band files)
# band_files = sorted(glob.glob('bands/*.tif'))
# bands = [rasterio.open(f) for f in band_files]
# meta = bands[0].meta.copy()
# meta.update(count=len(bands))
# stack = np.stack([b.read(1) for b in bands])
# with rasterio.open('stacked.tif','w',**meta) as dst:
#     dst.write(stack)

# Mosaic example (multiple tiles)
# tiles = [rasterio.open(t) for t in glob.glob('tiles/*.tif')]
# mosaic, out_trans = merge(tiles)
# out_meta = tiles[0].meta.copy()
# out_meta.update({'height': mosaic.shape[1], 'width': mosaic.shape[2], 'transform': out_trans})
# with rasterio.open('mosaic.tif','w',**out_meta) as dest:
#     dest.write(mosaic)

# Clip by AOI
# raster = rasterio.open('mosaic.tif')
# aoi = gpd.read_file('aoi.shp')
# geom = [aoi.geometry.unary_union.__geo_interface__]
# out_img, out_trans = mask(raster, geom, crop=True)
# show(out_img)

## 9. Classification: Unsupervised (KMeans) & Supervised (RandomForest)
The supervised example expects a training shapefile with a 'class' field.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Unsupervised KMeans (example using stacked image 'img' shaped (bands, rows, cols))
# img_ = img.copy()
# n_bands, rows, cols = img_.shape
# X = img_.reshape(n_bands, -1).T
# km = KMeans(n_clusters=5, random_state=42).fit(X)
# labels = km.labels_.reshape(rows, cols)
# plt.imshow(labels, cmap='tab10'); plt.title('KMeans classification'); plt.show()

# Supervised (Random Forest) - extract sample points from training shapefile
# training = gpd.read_file('training_samples.shp')
# coords = [(pt.x, pt.y) for pt in training.geometry]
# samples = [list(rasterio.open('stacked.tif').sample([c]))[0] for c in coords]
# X = np.array(samples)
# y = training['class'].values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)
# print('Train score:', rf.score(X_train, y_train), 'Test score:', rf.score(X_test, y_test))
# Predict full image
# img_2d = img_.reshape(n_bands, -1).T
# pred = rf.predict(img_2d).reshape(rows, cols)
# plt.imshow(pred, cmap='tab10'); plt.title('RF classification'); plt.show()

## 10. Change Detection (e.g., NDVI differencing)
Compute index for two dates and subtract to find gain/loss.

In [ ]:
# Example: NDVI change detection
# ndvi1 = rasterio.open('ndvi_2018.tif').read(1)
# ndvi2 = rasterio.open('ndvi_2024.tif').read(1)
# change = ndvi2 - ndvi1
# plt.imshow(change, cmap='RdBu'); plt.colorbar(); plt.title('NDVI change (2024-2018)'); plt.show()

## 11. Advanced Visualization
# Choropleth, classified maps, overlay vector+raster, colorbars, legend tips.

In [ ]:
# Example: Choropleth using GeoPandas
# shp = gpd.read_file('districts.shp')
# shp.plot(column='population_density', legend=True, cmap='OrRd')
# plt.title('Population Density')
# plt.show()

# Overlay raster and vector
# fig, ax = plt.subplots(figsize=(8,8))
# show(rasterio.open('mosaic.tif'), ax=ax)
# gpd.read_file('districts.shp').to_crs(rasterio.open('mosaic.tif').crs).boundary.plot(ax=ax, edgecolor='black')
# plt.show()

## 12. Notes & Next Steps
- Replace placeholder file paths with your file locations.
- Install required packages: `pip install rasterio geopandas scikit-image sklearn matplotlib seaborn earthpy` (use conda for complex deps).
- If you want, you can run each section step-by-step and save outputs.

---

**If this notebook looks good I will save it now as** `Python_RemoteSensing_GIS_Tutorial.ipynb`.